## The molecular dynamics and free energy calculation workflow
This jupyter notebook reads in an OpenMM-ready system prepared by `build.py`, runs an MD simulation (minimization, equilibration, and production runs), and then estimates the absolute ligand binding free energy using `MMPBSA.py`, which is distributed with `AmberTools`.

In [1]:
import time
import os
from string import Template
from typing import Tuple
import numpy as np
import pandas as pd
import MDAnalysis as mda
from openmm import *
from openmm.app import PDBFile, Topology, Simulation, DCDReporter, StateDataReporter, CheckpointReporter
import parmed as pmd

In [15]:
# In OpenMM, the trajectory is save as dcd format, while the Amber netcdf file is preferred in MMPBSA calculation.
# Therefore, we need to generate a netcdf trajectory file before proceeding to the MMPBSA calculation
cpptraj_nc_template = Template('''cpptraj -p ${outdir}/${complextop} << EOF
trajin ${outdir}/${dcd}
trajout ${outdir}/complex.nc netcdf
EOF
''')


def load_system(xml_path: str, pdb_path: str) -> Tuple[System, Topology, unit.Quantity]:
    """
    Load an OpenMM System from serialized XML and return (system, topology, positions).

    Parameters
    ----------
    xml_path : str
        Path to the system XML file
    pdb_path : str
        Path to the solvated PDB structure

    Returns
    -------
    system, topology, positions
    """
    # Deserialize system
    with open(xml_path, 'r') as f:
        system = XmlSerializer.deserialize(f.read())
    # Load topology and positions
    pdb = PDBFile(pdb_path)
    return system, pdb.topology, pdb.positions


def setup_simulation(system: System, topology: Topology, positions: unit.Quantity,
                     temperature: float = 300.0,
                     friction_coef: float = 1.0,
                     timestep_fs: float = 2.0,
                     pressure_bar: float = None,
                     platform_name: str = 'CPU',
                     checkpoint_file: str = None) -> Simulation:
    """
    Create and return an OpenMM Simulation object with Langevin integrator
    and optional barostat for NPT runs.

    Parameters
    ----------
    system : System
        The OpenMM System object
    topology : app.Topology
        The OpenMM toplogy object for the system
    positions : unit.Quantity
        The positions of all atoms in the topology. It is a list of Vec3
    temperature : float
        The simulation temperature in Kelvin
    friction_coef : float
        The collision rate in 1/ps. Used in Langevin integrator
    timestep_fs : float
        The integration timestep in femtoseconds
    pressure_bar : float
        If provided, adds a MonteCarloBarostat at this pressure (bar)
    platform_name : str
        OpenMM platform (e.g., 'CUDA', 'OpenCL', 'CPU'). The default is CPU.

    Returns
    -------
    simulation : Simulation
        The OpenMM Simulation object
    """
    # Add barostat (i.e. NPT simulation) if needed
    if pressure_bar is not None:
        system.addForce(MonteCarloBarostat(pressure_bar*unit.bar, temperature*unit.kelvin))

    # Create integrator
    integrator = LangevinIntegrator(
        temperature*unit.kelvin,
        friction_coef/unit.picosecond,
        timestep_fs*unit.femtosecond
    )

    # Select platform
    platform = Platform.getPlatformByName(platform_name)

    simulation = Simulation(topology, system, integrator, platform)
    simulation.context.setPositions(positions)

    # It is a restart instead of a fresh simulation if a checkpoint file exists
    if checkpoint_file is not None:
        # Load checkpoint
        with open(checkpoint_file, 'rb') as f:
            simulation.context.loadCheckpoint(f.read())
    return simulation


def add_reporters(simulation: Simulation,
                  traj_file: str = 'trajectory.dcd',
                  state_file: str = 'log.csv',
                  checkpoint_file: str = 'checkpoint.chk',
                  steps: int = 50000,
                  report_interval: int = 1000,
                  checkpoint_interval: int = 10000):
    """
    Attach DCD, state data, and checkpoint reporters.

    Parameters
    ----------
    simulation : Simulation
        The OpenMM Simulation object
    traj_file : str
        The trajectory file
    state_file : str
        The log file
    checkpoint_file: str
        The checkpoint file
    steps: int
        The simulation steps
    report_interval : int
        The step interval to save trajectory and write log file
    checkpoint_interval : int
        The step interval to write the checkpoint file

    Returns
    -------
    None
    """
    simulation.reporters.append(DCDReporter(traj_file, report_interval))
    simulation.reporters.append(
        StateDataReporter(
            state_file, report_interval,
            step=True, time=True,
            potentialEnergy=True, temperature=True,
            density=True, speed=True, remainingTime=True,
            totalSteps=steps, separator='\t'
        )
    )
    simulation.reporters.append(
        CheckpointReporter(checkpoint_file, checkpoint_interval)
    )


def add_restraints(system: System, positions: unit.Quantity,
                   atom_ids: list = [],
                   force_constant: float = 1000.0) -> Tuple[System, int]:
    """
    Apply harmonic positional restraints to heavy atoms of RNA and ligands.
    Returns the force index added to the system.

    Parameters
    ----------
    system : System
        The OpenMM System object
    positions : unit.Quantity
        The positions of all atoms in the topology. It is a list of Vec3
    atom_ids : list
        A list of the atom IDs to be restrained
    force_constant : float
        The force constant of the harmonic restraint in kcal/(mol nm^2)

    Returns
    -------
    system : System
        The updated OpenMM System object
    force_index : int 
        The Index of the harmonic restraint force
    """
    restraint = CustomExternalForce(
        "0.5*k*((x-x0)^2 + (y-y0)^2 + (z-z0)^2)"
    )
    restraint.addGlobalParameter('k', force_constant * unit.kilojoule_per_mole / unit.nanometer**2)
    restraint.addPerParticleParameter('x0')
    restraint.addPerParticleParameter('y0')
    restraint.addPerParticleParameter('z0')

    coords = np.array([[p.x, p.y, p.z] for p in positions])  # in nm
    for idx in atom_ids:
        x0, y0, z0 = coords[idx]
        restraint.addParticle(idx, [x0, y0, z0])
    force_index = system.addForce(restraint)
    return system, force_index
    

def run_equilibration(simulation: Simulation,
                      force_index: int,
                      temperature: float = 300.0,
                      equil_steps: int = 100000):
    """
    Run two equilibration stages:
    1) with positional restraints on RNA & ligand heavy atoms
    2) without restraints

    Parameters
    ----------
    simulation : Simulation
        The OpenMM Simulation object
    force_index : int 
        The Index of the harmonic restraint force
    temperature : float
        The simulation temperature in Kelvin
    equil_steps : int
        The equilibration steps

    Returns
    -------
    None
    """
    time_start = time.perf_counter()
    # Stage 1: restrained
    print('Initial system energy', simulation.context.getState(getEnergy=True).getPotentialEnergy())
    print('Minimizing...')
    simulation.minimizeEnergy()
    print('Minimized system energy', simulation.context.getState(getEnergy=True).getPotentialEnergy())
    print(f"The first equilibration step: {equil_steps} steps with restraints")
    # Initialize velocities
    simulation.context.setVelocitiesToTemperature(temperature*unit.kelvin)
    simulation.context.setStepCount(0)
    simulation.step(equil_steps)

    # Stage 2: unrestrained
    # Remove restraint force
    simulation.system.removeForce(force_index)
    simulation.context.reinitialize(preserveState=True)
    simulation.context.setPositions(simulation.context.getState(getPositions=True).getPositions())
    
    print(f"The second equilibration step: {equil_steps} steps without restraints")
    simulation.context.setStepCount(0)
    simulation.step(equil_steps)
    time_end = time.perf_counter()
    print(f"Total equilibration time: {(time_end - time_start)/60:.1f} minutes")


def run_production(simulation: Simulation, production_steps: int):
    """
    Run production MD for a given number of steps.

    Parameters
    ----------
    simulation : Simulation
        The OpenMM Simulation object
    production_steps : int
        The production steps
    
    Returns
    -------
    None
    """
    time_start = time.perf_counter()
    print(f"Running production: {production_steps} steps")
    simulation.context.setStepCount(0)
    simulation.step(production_steps)
    time_end = time.perf_counter()
    print(f"Total production time: {(time_end - time_start)/60:.1f} minutes")


# 1. Minimization and equilibration runs

In [9]:
# Load the system assembled by `build.py`
system, topo, pos = load_system('output/1O15_system.xml', 'output/1O15_solvated.pdb')

In [10]:
# Add harmonic restraint to heavy atoms of RNA and ligand
u = mda.Universe('output/1O15_solvated.pdb')
heavy_atoms = u.select_atoms('(nucleic or resname TEP) and not name H*')
atom_ids = heavy_atoms.ids-1
system, restrain_idx = add_restraints(system, pos, atom_ids, 100)

In [11]:
# Set up and simulation
sim = setup_simulation(system, topo, pos,
                       temperature=300, friction_coef=1.0,
                       timestep_fs=1.0, pressure_bar=1.0)

add_reporters(sim, steps=10000, traj_file='output/equil.dcd', state_file='output/equil.log', checkpoint_file='output/equil.chk',
              report_interval=1000, checkpoint_interval=10000)

In [14]:
# Two-stage equilibration: 50k steps restrained, 50k unrestrained
run_equilibration(sim, restrain_idx, equil_steps=10000)

Initial system energy -405413.66638635204 kJ/mol
Minimizing...
Minimized system energy -459218.4279758284 kJ/mol
The first equilibration step: 10000 steps with restraints
The second equilibration step: 10000 steps without restraints
Total equilibration time: 4.5 minutes


# 2. Production run

In [16]:
# Load the system assembled by `build.py`
system, topo, pos = load_system('output/1O15_system.xml', 'output/1O15_solvated.pdb')
# Set up and simulation
sim = setup_simulation(system, topo, pos,
                       temperature=300, friction_coef=1.0,
                       timestep_fs=2.0, pressure_bar=1.0,
                       checkpoint_file='output/equil.chk')
add_reporters(sim, steps=500000, traj_file='output/prod.dcd', state_file='output/prod.log', checkpoint_file='output/prod.chk',
              report_interval=1000, checkpoint_interval=10000)

In [17]:
# Carry out production run
run_production(sim, production_steps=500000)

Running production: 500000 steps
Total production time: 170.4 minutes


# 3. Binding free energy calculation using MMPBSA

In [19]:
# Convert the dcd file to netcdf file
cpptraj_nc = cpptraj_nc_template.substitute(outdir='output', complextop='1O15_solvated.prmtop',
                                            dcd='prod.dcd')
os.system(cpptraj_nc)


CPPTRAJ: Trajectory Analysis. V6.24.0 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 07/11/25 10:11:46
| Available memory: 119.263 GB

	Reading 'output/1O15_solvated.prmtop' as Amber Topology
	Radius Set: 0
INPUT: Reading input from 'STDIN'
  [trajin output/prod.dcd]
	Reading 'output/prod.dcd' as Charmm DCD
	Symmetric shape matrix detected.
  [trajout output/complex.nc netcdf]
	Writing 'output/complex.nc' as Amber NetCDF
---------- RUN BEGIN -------------------------------------------------

PARAMETER FILES (1 total):
 0: 1O15_solvated.prmtop, 24020 atoms, 7728 res, box: Cubic, 7696 mol, 7620 solvent

INPUT TRAJECTORIES (1 total):
 0: 'prod.dcd' is a CHARMM DCD file (coords) Little Endian 32 bit, Parm 1O15_solvated.prmtop (Cubic box) (reading 500 of 500)
  Coordinate processing will occur on 500 frames.

OUTPUT TRAJECTORIES (1 total):
  'complex.nc' (500 frames) is a NetCDF (NetCDF3) AMBER trajectory

BEGIN TRAJECTORY PROCESSING:
......

0

In [21]:
# Write the MMPBSA input file
# note: 1. save the intermediate trajectories as netcdf files (netcdf=1) to save storage space
#       2. explicitly specify the atoms to be stripped and the ligand resname
#       3. calculate binding free energy every 5 frames to both save time and reduce auto-correlation
with open('output/mmpbsa.in', 'w') as w:
    w.write(f'''&general
  startframe=1, endframe=500, interval=5,
  verbose=1,
  netcdf=1,
  strip_mask=":HOH:NA:CL",
  ligand_mask=":TEP"
/
&gb
  igb=5, saltcon=0.150
/
&pb
  istrng=0.150,
/
&decomp
  idecomp=1,
/''')

In [35]:
# Run MMPBSA calculation
os.system(f'''cd output
MMPBSA.py -O \
  -i mmpbsa.in \
  -sp 1O15_solvated.prmtop \
  -cp complex_strip.prmtop \
  -rp receptor.prmtop \
  -lp ligand.prmtop \
  -y complex.nc
cd ../''')

# (Optional) 4. Rerun the pipeline with ANI-2x potentials

This section reruns the MD/MMPBSA pipeline but the ligand's vdW and bonded terms are parameterized by ANI-2x.

In [36]:
from openmmml import MLPotential
import torchani

In [28]:
def parameterize_ligand_with_ani2x(system: System, topology: Topology, ligname: str) -> System:
    """
    Replace all intra-ligand GAFF2 bonded & vdW terms with ANI-2x.

    Parameters
    ----------
    system : System
        The OpenMM System object
    topology : Topology
        The OpenMM Topology object
    ligand_resname : str
        The residue name of the ligand

    Returns
    -------
    system : System
        The modified OpenMM System object, with ANI-2x force added
    """
    # Identify ligand atom indices
    ligand_atoms = [atom.index for atom in topology.atoms()
                    if atom.residue.name == ligname]
    # Build an MLPotential using ANI-2x, and mix it into the rest of the system
    potential = MLPotential('ani2x')
    system = potential.createMixedSystem(topology, system, ligand_atoms)
    return system

In [34]:
# load classical
sys, topo, pos = load_system('output/1O15_system.xml', 'output/1O15_solvated.pdb')
# build hybrid
hybrid_sys = parameterize_ligand_with_ani2x(sys, topo, ligname='TEP')
# Add harmonic restraint to heavy atoms of RNA and ligand
u = mda.Universe('output/1O15_solvated.pdb')
heavy_atoms = u.select_atoms('(nucleic or resname TEP) and not name H*')
atom_ids = heavy_atoms.ids-1
hybrid_sys, restrain_idx = add_restraints(hybrid_sys, pos, atom_ids, 100)

In [32]:
# Set up and simulation
sim = setup_simulation(hybrid_sys, topo, pos,
                       temperature=300, friction_coef=1.0,
                       timestep_fs=1.0, pressure_bar=1.0)

add_reporters(sim, steps=1000, traj_file='output/ani_equil.dcd', state_file='output/ani_equil.log', 
              checkpoint_file='output/ani_equil.chk', report_interval=100, checkpoint_interval=50)

In [42]:
# Two-stage equilibration: 1k steps restrained, 1k unrestrained
run_equilibration(sim, restrain_idx, equil_steps=1000)

Initial system energy -1845070.465569044 kJ/mol
Minimizing...
Minimized system energy -459218.4279758284 kJ/mol
The first equilibration step: 1000 steps with restraints
The second equilibration step: 1000 steps without restraints
Total equilibration time: 100.4 minutes


In [38]:
# Load the system assembled by `build.py`
sys, topo, pos = load_system('output/1O15_system.xml', 'output/1O15_solvated.pdb')
# build hybrid
hybrid_sys = parameterize_ligand_with_ani2x(sys, topo, ligname='TEP')

In [39]:
# Set up and simulation
sim = setup_simulation(hybrid_sys, topo, pos,
                       temperature=300, friction_coef=1.0,
                       timestep_fs=2.0, pressure_bar=1.0,
                       checkpoint_file='output/ani_equil.chk')
add_reporters(sim, steps=2000, traj_file='output/ani_prod.dcd', state_file='output/ani_prod.log', 
              checkpoint_file='output/ani_prod.chk', report_interval=100, checkpoint_interval=1000)

In [41]:
# Production run
run_production(sim, production_steps=2000)

Running production: 2000 steps
Total production time: 33.5 minutes
